In [1]:
# Basic environment modules
import numpy as np
import pandas as pd
import psycopg2 as pg # PostgreSQL module
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import font_manager
import matplotlib.font_manager as fm

# Data analysis modules
from CoolProp.CoolProp import PropsSI
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from scipy.integrate import ode
from sqlalchemy import create_engine

# User-defined libraries
from Model import *
from PhysicalProperty import *
from StructuredQuery import *
from Numeric import *
from Authentication import *

print("Import modules successfully.")


# load class instances
sql = StructuredQuery()
pro = PhysicalProperty()
mod = Model()
oau = Authentication()

Import modules successfully.
Construct connection string and update
SQL library is successfully started.
Property_OSV is successfully started.
Models of boiling heat transfer is successfully started.
Authentication is successfully started.


In [2]:
# Connect to DB Server and get data
# Load json file (Security)
json_parse = oau.get_apikey(json_filename='key.json')
json_res = pd.json_normalize(json_parse['sql'])

# Set query from DB
sql = "SELECT * FROM CHF.raw_database WHERE refri IN ('H2O') AND idx != '40' AND idx != '41' AND p <= 220"

# Connect DB server
#(conn, db_engine) = sql.connect(json_res["host"][1], json_res["dbname"][1], json_res["user"][1], json_res["port"][1], json_res["password"][1], json_res["service"][1]) #postgreSQL
connect = pys.connect(host=json_res["host"][1], user = json_res["user"][1], password = json_res["password"][1], cursorclass=pys.cursors.DictCursor)
cur = connect.cursor()
cur.execute(sql)


result = cur.fetchall()
raw_tb = pd.DataFrame(result)

cur.close()
del result # delete temp file
del sql # delete quiry log

print("Loading database completed. data size: {}".format(len(raw_tb)))

Loading database completed. data size: 10531


In [3]:
# Calculation of physical properties

# if physical properties can't calculated, the other step is need (interpolation based on the physical properties table) = Sodium
prop_tb = raw_tb.copy()

"""
R113, R114 can't use viscosity model -> re-mapping these models based on interpolation tables
R12 Pressure range : 0.242551~4.1361e+06 Pa (P, Q Flash)
H2O Pressure range : 611.655~2.2064e+07 Pa (P, Q Flash)
"""

# start the timer
start_time = time.time()
print('START TIME :',str(datetime.now())[10:19] )

# compute physical properties
for i in range(0, len(raw_tb)):
    # calculated physical properties at saturation point
    prop_tb.loc[i,'tsat'] = round(PropsSI('T', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'pcrit'] = round(PropsSI(raw_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'rdcp'] = round(raw_tb.loc[i,'p']/prop_tb.loc[i,'pcrit'],12)
    prop_tb.loc[i,'rhof'] = round(PropsSI('D', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'rhov'] = round(PropsSI('D', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'muf'] = round(PropsSI('V', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'muv'] = round(PropsSI('V', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'hfo'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'hvo'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'lam'] = round((prop_tb.loc[i, 'hvo'] - prop_tb.loc[i,'hfo']),12) # [kJ/kg]

    # calculate physical propoerties of water at same conditions
    prop_tb.loc[i,'wtsat'] = round(PropsSI('T', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wpcrit'] = round(PropsSI(raw_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'wrdcp'] = round(raw_tb.loc[i,'p']/prop_tb.loc[i,'pcrit'],12)
    prop_tb.loc[i,'wrhof'] = round(PropsSI('D', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wrhov'] = round(PropsSI('D', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, 'H2O'),12)
    prop_tb.loc[i,'wmuf'] = round(PropsSI('V', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wmuv'] = round(PropsSI('V', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, 'H2O'),12)
    prop_tb.loc[i,'whfo'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O')*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'whvo'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, 'H2O')*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'wlam'] = round((prop_tb.loc[i, 'whvo'] - prop_tb.loc[i,'whfo']),12) # [kJ/kg]

    # calculate flow parameters
    if pd.isna(raw_tb.loc[i, 'v']):
        prop_tb.loc[i, 'v'] = round(raw_tb.loc[i, 'g'] / prop_tb.loc[i, 'rhof'], 12) # [m/s]
    else:
        if pd.isna(raw_tb.loc[i, 'g']):
            prop_tb.loc[i, 'g'] = round(raw_tb.loc[i, 'v']*prop_tb.loc[i, 'rhof'], 12) # [kg/m2s]
    prop_tb.loc[i, 'cpf']   = round(PropsSI('C', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'cpv']   = round(PropsSI('C', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'sigma'] = round(PropsSI('I', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12) # [N/m]
    prop_tb.loc[i, 'kf'] = round(PropsSI('L', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12) # [W/m/K]
    prop_tb.loc[i, 'kv'] = round(PropsSI('L', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12) # [W/m/K]

    prop_tb.loc[i, 'wcpf']   = round(PropsSI('C', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O'),12) # [J/kgK]
    prop_tb.loc[i, 'wcpv']   = round(PropsSI('C', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, 'H2O'),12) # [J/kgK]
    prop_tb.loc[i, 'wsigma'] = round(PropsSI('I', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O'),12) # [N/m]
    prop_tb.loc[i, 'wkf'] = round(PropsSI('L', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, 'H2O'),12) # [W/m/K]
    prop_tb.loc[i, 'wkv'] = round(PropsSI('L', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, 'H2O'),12) # [W/m/K]

    # enthin is empty
    # Fill Na related to inlet subcooling
    if pd.isna(raw_tb.loc[i, 'enthin']):
        # enthin is NaN
        if pd.isna(raw_tb.loc[i, 'tin']):
            # enthin, tin is NaN
            if pd.isna(raw_tb.loc[i, 'xi']):
                # enthin, tin, xi is NaN
                if pd.isna(raw_tb.loc[i, 'degtin']):
                    # enthin, tin, xi, and degtin is NaN
                    print("{:d} The data of inlet conditions should be defined.".format(i))
                else:
                    # enthin, tin, and xi is NaN
                    prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin']
                    prop_tb.loc[i, 'hin'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tin'], 'P', raw_tb.loc[i, 'p'] * 1e5, raw_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
                    prop_tb.loc[i, 'enthin'] = prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'] # [kJ/kg]
                    prop_tb.loc[i, 'xi'] = -prop_tb.loc[i, 'enthin'] / prop_tb.loc[i, 'lam'] #[-]
            else:
                # enthin, tin is NaN
                prop_tb.loc[i, 'xi'] = raw_tb.loc[i, 'xi'] #[-]
                prop_tb.loc[i, 'enthin'] = - prop_tb.loc[i, 'xi'] * prop_tb.loc[i, 'lam'] # [kJ/kg]
                prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - raw_tb.loc[i, 'enthin'], 12) # [kJ/kg]
                prop_tb.loc[i, 'degtin'] = prop_tb.loc[i, 'enthin']*10**3/prop_tb.loc[i, 'cpf'] # degC
                prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] # degC
        else:
            # enthin is Nan
            prop_tb.loc[i, 'tin'] = raw_tb.loc[i, 'tin']
            prop_tb.loc[i, 'hin'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tin'] +273.15, 'P', raw_tb.loc[i, 'p'] * 1e5, raw_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
            prop_tb.loc[i, 'enthin'] = prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'] # [kJ/kg]
            prop_tb.loc[i, 'xi'] = -prop_tb.loc[i, 'enthin'] / prop_tb.loc[i, 'lam'] #[-]
            prop_tb.loc[i, 'degtin'] = prop_tb.loc[i, 'enthin']*10**3/prop_tb.loc[i, 'cpf'] # degC
    else:
        if pd.isna(raw_tb.loc[i, 'xi']):
            prop_tb.loc[i, 'xi'] = -prop_tb.loc[i, 'enthin'] / prop_tb.loc[i, 'lam'] #[-]
            prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - raw_tb.loc[i, 'enthin'], 12) # [kJ/kg]
            prop_tb.loc[i, 'degtin'] = prop_tb.loc[i, 'enthin']*10**3/prop_tb.loc[i, 'cpf'] # degC
            prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] # degC
        else:
            prop_tb.loc[i, 'xi'] = raw_tb.loc[i, 'xi'] #[-]
            prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - raw_tb.loc[i, 'enthin'], 12) # [kJ/kg]
            prop_tb.loc[i, 'degtin'] = prop_tb.loc[i, 'enthin']*10**3/prop_tb.loc[i, 'cpf'] # degC
            prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] # degC

    if pd.isna(raw_tb.loc[i, 'xe']):
        prop_tb.loc[i, 'xe'] = round(pro.cal_xe(prop_tb.loc[i, 'q'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'enthin'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam']), 12)
    else:
        prop_tb.loc[i, 'xe'] = round(raw_tb.loc[i, 'xe'], 12) # Copying raw data  #[-]


""" re-check exit enthalpy
    if pd.isna(raw_tb.loc[i, 'xe']):
        print(i)
        # xe is NaN
        if pd.isna(raw_tb.loc[i, 'enthout']):
            # xe and enthout is NaN
            prop_tb.loc[i, 'tout'] = round(raw_tb.loc[i, 'tout'],12) # this value must exist
            #prop_tb.loc[i, 'enthout'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tout'] +273.15, 'P', raw_tb.loc[i, 'p'] * 1e5, raw_tb.loc[i, 'refri'])*1e-3 - prop_tb.loc[i, 'hin'], 12) #[kJ/kg]
            prop_tb.loc[i, 'xe'] = round(pro.cal_xe(raw_tb.loc[i, 'q'], raw_tb.loc[i, 'doi'], raw_tb.loc[i, 'dio'], raw_tb.loc[i, 'geo'], raw_tb.loc[i, 'hs'], raw_tb.loc[i, 'g'], prop_tb.loc[i, 'enthin'], raw_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam']), 12)            
        else:
            # xe is NaN
            prop_tb.loc[i, 'enthout'] = round(raw_tb.loc[i, 'enthout'], 12) # Copying raw data  #[-]
            prop_tb.loc[i, 'xe'] = round(-raw_tb.loc[i, 'enthout']/prop_tb.loc[i, 'lam'], 12)  #[-]
            prop_tb.loc[i, 'tout'] = round(prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'enthout']*10**3/prop_tb.loc[i, 'cpf'], 12) # degC
            
    else:
        prop_tb.loc[i, 'xe'] = round(raw_tb.loc[i, 'xe'], 12) # Copying raw data  #[-]
        prop_tb.loc[i, 'enthout'] = - prop_tb.loc[i, 'xe'] * prop_tb.loc[i, 'lam'] # [kJ/kg]
        prop_tb.loc[i, 'tout'] = round(prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'enthout']*10**3/prop_tb.loc[i, 'cpf'], 12) # degC
"""

engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# Export results of physical property to MariaDB
prop_tb.to_sql('prop_chf_tb', engine, if_exists='replace')

del engine

# stop the timer
end_time1 = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time1 = end_time1 - start_time

# print the elapsed time
print("Elapsed time to compute the physical properties: {:.2f} seconds".format(elapsed_time1))

START TIME :  13:25:08
END TIME :  13:26:54
Elapsed time to compute the physical properties: 106.53 seconds


In [4]:
# start the timer
print('START TIME :',str(datetime.now())[10:19] )

for i, row in prop_tb.iterrows():
    # Check heat balance
    prop_tb.loc[i,'hval'] = round(prop_tb.loc[i,'xi'] + 4*prop_tb.loc[i, 'q']*10**3*prop_tb.loc[i,'lh']/(prop_tb.loc[i, 'dh']*prop_tb.loc[i,'g']*prop_tb.loc[i,'lam']),6)
    prop_tb.loc[i,'param_hval'] = round(np.abs(prop_tb.loc[i ,'xe']-prop_tb.loc[i, 'hval'])/prop_tb.loc[i, 'hval']*100, 6)
    if prop_tb.loc[i, 'param_hval'] < 10:
        prop_tb.loc[i, 'yn_hval'] = 100
    else:
        prop_tb.loc[i, 'yn_hval'] = 110

    # Calculate dimensionless number
    prop_tb.loc[i, 'pe'] = round(
        pro.cal_pe(prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 're'] = round(pro.cal_re(prop_tb.loc[i, 'g'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'muf']), 6)
    prop_tb.loc[i, 'we'] = round(
        pro.cal_we(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'v'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'bd'] = round(
        pro.cal_bd(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'pr'] = round(pro.cal_pr(prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 'prv'] = round(pro.cal_pr(prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'kv']), 6)
    prop_tb.loc[i, 'ca'] = round(
        pro.cal_ca(prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'v'], prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'rhof']), 6)
    prop_tb.loc[i, 'lc'] = round(np.sqrt(prop_tb.loc[i, 'sigma']/(9.8*(prop_tb.loc[i, 'rhof']-prop_tb.loc[i, 'rhov']))), 6)

    if prop_tb.loc[i, 'geo'] == 'A':
        prop_tb.loc[i, 'dl'] = prop_tb.loc[i, 'dh']-2*prop_tb.loc[i, 'lc']
    elif prop_tb.loc[i, 'geo'] == 'R':
        if prop_tb.loc[i, 'dio'] < prop_tb.loc[i, 'lc']:
            prop_tb.loc[i, 'dl'] = prop_tb.loc[i, 'dh']/prop_tb.loc[i, 'ar']
        else:
            prop_tb.loc[i, 'dl'] = ((2*prop_tb.loc[i, 'doi']*(prop_tb.loc[i, 'dio'] - prop_tb.loc[i, 'lc']))/(prop_tb.loc[i, 'doi']+prop_tb.loc[i, 'dio'] - prop_tb.loc[i, 'lc']))/prop_tb.loc[i, 'ar']
    else:
        prop_tb.loc[i, 'dl'] = prop_tb.loc[i, 'dh']+2*prop_tb.loc[i, 'lc']

# stop the timer
end_time2 = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time2 = end_time2 - end_time1

# print the elapsed time
print("Elapsed time to compute the geometrical or dimensionless parameters: {:.2f} mins".format(elapsed_time2/60))


START TIME :  13:26:54
END TIME :  13:27:17
Elapsed time to compute the geometrical or dimensionless parameters: 0.37 mins


In [5]:
# set iniital calculation method
m_idx = 1
cal_idx = 3
mod_idx = 'Deng'

# create log table 
log_tb = pd.DataFrame()

# set global parameters
global j
j = 0
tolerance = 0.01

# check start time
print('START TIME : ',str(datetime.now())[10:19])

for i, row in prop_tb.iterrows():
    if m_idx == 1:
        prop_tb.loc[i, 'mth_xosv'] = 'SZ'
    else:
        prop_tb.loc[i, 'mth_xosv'] = 'Levy'
    
    cnt = 1
    # Preparing alpha and gamma
    prop_tb.loc[i, 'log_id'] = int(i) # Set foreign key to log-table
    prop_tb.loc[i, 'alpha'] = round(1.669-6.544*(prop_tb.loc[i, 'rdcp']/prop_tb.loc[i, 'dh']-0.448)**2,16)
    prop_tb.loc[i, 'gamma'] = round(0.06523 + (0.1045/(np.sqrt((2*np.pi)*(np.log(prop_tb.loc[i, 'rdcp']))**2))) * np.exp(-5.413*((np.log(prop_tb.loc[i, 'rdcp'])+0.4537)**2/(np.log(prop_tb.loc[i, 'rdcp'])**2))),16)

    while 1:
        # Prepare : log-table
        log_tb.loc[j, 'run_id'] = int(i)
        log_tb.loc[j, 'org_xi'] = prop_tb.loc[i, 'xi']
        log_tb.loc[j, 'org_xe'] = prop_tb.loc[i, 'xe']
        log_tb.loc[j, 'org_q'] = prop_tb.loc[i, 'q']
        log_tb.loc[j, 'yn_hval'] = prop_tb.loc[i, 'yn_hval']
        
        # Ref: Original algorithm
            # calculate initial Xt from qc correlation using qexp
            # calculate Xo using qexp = - StGlamXo
            # calculate Xt from rate equation
            # compare oldXt vs. new Xt

        # Step 0: Initialize Xt when the first iteration step
        if cnt ==1:
            log_tb.loc[j, 'old_xt'] = 0.001
        else:
            pass
        # Step 2: Calculate qc,old
        if mod_idx == 'Deng':
            log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'old_qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], log_tb.loc[j, 'old_xt'])
        else:
            log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'old_qcal'] = mod.calCHFJeong(prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'lam'], log_tb.loc[j, 'old_xt'])
        
        # Step 3: Calculate old_xosv
        if m_idx == 1:
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_SZ(log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
        else:
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'g'], log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'v'])

        # Step 4: Calculate old_xe
        log_tb.loc[j, 'old_xe'] = pro.cal_xe(log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'enthin'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 1)

        # Step 2: Calculate old_xt (initialization)
        log_tb.loc[j, 'new_xt'] = round(mod.cal_xt(log_tb.loc[j, 'org_xi'], log_tb.loc[j, 'old_xosv'], log_tb.loc[j, 'org_xe'], log_tb.loc[j, 'old_xe']),6)

        # Step 2: Calculate qc,old
        if mod_idx == 'Deng':
            log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'new_qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], log_tb.loc[j, 'new_xt'])
        else:
            log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'new_qcal'] = mod.calCHFJeong(prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'lam'], log_tb.loc[j, 'new_xt'])

        # Step 4: Loop control
        cnt += 1 # Increase limitation

        # Step 5: Valudation
        val = round(abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt']),12)

        if val <= tolerance:
            # Step 2: Calculate old_xt (initialization)
            prop_tb.loc[i, 'cal_xt'] = round(float(log_tb.loc[j, 'new_xt']),6)
            prop_tb.loc[i, 'cal_xe'] = round(log_tb.loc[j, 'old_xe'], 4)
            prop_tb.loc[i, 'cal_xosv'] = round(log_tb.loc[j, 'old_xosv'], 4)
            prop_tb.loc[i, 'converged'] = 100
            prop_tb.loc[i, 'alpha'] = log_tb.loc[j, 'alpha']
            prop_tb.loc[i, 'gamma'] = log_tb.loc[j, 'gamma']
            prop_tb.loc[i, 'zxt'] = round(float(log_tb.loc[j, 'zxt']), 6)
            prop_tb.loc[i, 'qcal'] = log_tb.loc[j, 'new_qcal']
            prop_tb.loc[i, 'qval'] = round((log_tb.loc[j, 'new_qcal'] - log_tb.loc[j, 'org_q'])/log_tb.loc[j, 'org_q'], 4)
            print("Data run number : {:d} is converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'q'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
            j += 1
            del cnt
            break
        else:
            if cnt > 1000:
                prop_tb.loc[i, 'cal_xt'] = round(float(log_tb.loc[j, 'new_xt']),6)
                prop_tb.loc[i, 'cal_xe'] = log_tb.loc[j, 'old_xe']
                prop_tb.loc[i, 'cal_xosv'] = log_tb.loc[j, 'old_xosv']
                prop_tb.loc[i, 'converged'] = 110
                prop_tb.loc[i, 'alpha'] = log_tb.loc[j, 'alpha']
                prop_tb.loc[i, 'gamma'] = log_tb.loc[j, 'gamma']
                prop_tb.loc[i, 'zxt'] = round(float(log_tb.loc[j, 'zxt']), 6)
                prop_tb.loc[i, 'qcal'] = log_tb.loc[j, 'new_qcal']
                prop_tb.loc[i, 'qval'] = ((log_tb.loc[j, 'new_qcal'] - log_tb.loc[j, 'org_q'])/log_tb.loc[j, 'org_q'])
                print("Data run number : {:d} isn't converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'q'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                j += 1
                del cnt
                break
            else:
                # Step 5: Recurrsive state
                log_tb.loc[j+1, 'old_xt'] = 0.995*log_tb.loc[j, 'old_xt'] + 0.005*log_tb.loc[j, 'new_xt']
                #log_tb.loc[j+1, 'old_qcal'] = 0.1* log_tb.loc[j, 'new_qcal'] + 0.9 * log_tb.loc[j, 'old_qcal']
                #log_tb.loc[j+1, 'old_xe'] = log_tb.loc[j, 'old_xe']
                j += 1
                continue

# stop the timer
end_time3 = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time3 = end_time3 - end_time2

# print the elapsed time
print("Elapsed time to compute the physical properties: {:.2f} mins".format(elapsed_time3/60))

# delete parameters
del i, j, tolerance

START TIME :   13:27:17
Data run number : 0 is converged. Xosv: -0.0296, Xe: 0.2770, Xt: 0.3235, qexp: 4.3218, qcal: 4.7710, and qval: 0.10
Data run number : 1 is converged. Xosv: -0.0331, Xe: 0.1670, Xt: 0.1953, qexp: 4.9212, qcal: 5.3365, and qval: 0.08
Data run number : 2 is converged. Xosv: -0.0354, Xe: 0.1090, Xt: 0.1320, qexp: 5.2682, qcal: 5.7135, and qval: 0.08
Data run number : 3 is converged. Xosv: -0.0393, Xe: 0.0510, Xt: 0.0697, qexp: 5.8675, qcal: 6.3416, and qval: 0.08
Data run number : 4 is converged. Xosv: -0.0438, Xe: 0.0250, Xt: 0.0346, qexp: 7.0663, qcal: 7.0849, and qval: 0.00
Data run number : 5 is converged. Xosv: -0.0507, Xe: 0.0120, Xt: 0.0122, qexp: 9.6846, qcal: 8.2055, and qval: -0.15
Data run number : 6 is converged. Xosv: -0.0566, Xe: 0.0080, Xt: 0.0046, qexp: 12.5868, qcal: 9.1697, and qval: -0.27
Data run number : 7 is converged. Xosv: -0.0664, Xe: 0.0070, Xt: 0.0005, qexp: 19.2745, qcal: 10.7747, and qval: -0.44
Data run number : 8 is converged. Xosv: -0

In [ ]:
# create engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# export the log- and result-table to MariaDB
log_tb.to_sql('new_log_chf_tb', engine, if_exists='replace')
prop_tb.to_sql('new_res_chf_tb', engine, if_exists='replace')

# delete parameter
del engine

In [ ]:
log_tb.loc[log_tb['run_id'] ==2267]

In [ ]:
log_tb.loc[:, ['run_id','old_xosv','old_qcal','org_q', 'new_qcal']]


In [ ]:
# Kmeans Algorithm
# Preparing converged dataset
val_tb = prop_tb[(prop_tb['converged']==100) & (prop_tb['yn_hval'] == 100) & (prop_tb['refri'] == 'H2O') & (prop_tb['dh'] != np.nan)].copy()

val_tb['Y_qcal'] = val_tb.apply(lambda x: ((x['q']*10**3*x['dh']/x['lam'])), axis = 1) # Y value
val_tb['X_zxt'] = val_tb.apply(lambda x: (x['cal_xt']), axis = 1) # X value

# Preparing linear form of CHF correlation
X = val_tb[['rdcp']]

from sklearn.preprocessing import QuantileTransformer
# Scaling
scaler = QuantileTransformer()
X_scaled = scaler.fit_transform(X)

# Calculate density of each sample
no_k = 40
wcss = []

for k in range(1, no_k):
    kmeans = KMeans(n_clusters=k, random_state=0, max_iter=1000, n_init=50)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Plot the WCSS values as a function of the number of clusters
plt.plot(range(1, no_k), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

from kneed import KneeLocator
# Find the elbow point in the WCSS curve
kneedle = KneeLocator(range(1, no_k), wcss, curve = 'convex', direction='decreasing')
elbow_point = kneedle.elbow
print(elbow_point)

# Run K-means clustering for elbow point
kmeans = KMeans(n_clusters=elbow_point, random_state=0, max_iter=1000, n_init=1).fit(X, sample_weight=1)

# Extract the labels (cluster assignments) and centroids
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Add the labels and centroids to the original DataFrame
val_tb['cluster'] = labels
val_tb['centroid_distance'] = kmeans.transform(X).min(axis=1)

# Create a 1X2 grid of subplots
fig, axs = plt.subplots(1, 2, figsize=(15,5))
fig.subplots_adjust(hspace=0.2, wspace=0.2)

# Set configuration of figure
axs[0].scatter(val_tb['rhof']/val_tb['rhov'], val_tb['q'], c=val_tb['cluster'], cmap ='gist_rainbow')
axs[1].scatter(val_tb['X_zxt'], val_tb['Y_qcal'], c=val_tb['cluster'], cmap ='gist_rainbow')
plt.xlabel('Pressure [bar]')
plt.ylabel('ln(qc*sqrt(d))')
plt.show()

In [ ]:
# DBSCAN algorithm
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Preparing linear form of CHF correlation
X = val_tb[['rdcp']]

scaler = QuantileTransformer()
X_scaled = scaler.fit_transform(X)

# Run DBSCAN algorithm
dbscan = DBSCAN(eps=0.00001, min_samples = 5)
labels = dbscan.fit_predict(X)

# Plot the results
plt.scatter(X['rdcp'], val_tb['q'], c=labels, cmap ='gist_rainbow')
plt.show()

In [ ]:
# Clustering grouping and calculation for loop

# normalize the dataframe using MinMax Scaler
def cal_mnmx(df, cols):
    # create a MinMaxScaler object
    scaler = MinMaxScaler()

    # fit the scaler to the dataframe
    scaler.fit(df[cols])

    # transform the selected columns of the dataframe using the scaler
    res = pd.DataFrame(scaler.transform(df[cols]), columns=cols)

    return res

from sklearn.metrics import r2_score as r2
# define a function to perform linear regression on a group
def cal_lr(df):
    # create a linear regression object
    model = LinearRegression()
    cluster_value = len(df['cluster'])
    X = df[['X_zxt']].values.reshape(-1,1)
    y = df['Y_qcal'].values.reshape(-1,1)

    # fit the model to the data
    res_model = model.fit(X, y)
    r2_score=r2(y, res_model.predict(X))

    # return the slop and intercept of the regression line
    return pd.Series({'no': cluster_value, 'g_alpha': res_model.intercept_[0], 'g_gamma': res_model.coef_[0][0], 'r2_score': r2_score})

# group the DataFrame by the 'cluster' column
#val_group = val_tb.groupby('cluster').apply(cal_lr)
val_group_tb = val_tb.groupby('cluster').apply(cal_lr).reset_index()
val_group_tb

In [ ]:
# concatenate the two dataframe along rows
result = pd.merge(val_tb, val_group_tb, left_on='cluster', right_on = val_group_tb['cluster'], how='outer').copy()
#result2 = pd.merge(result, tmp_res, on = result.index, how='outer').copy()

# 이 부분은 함수로 데이터프레임에 적용할 수 있도록 수정해야 함.
lr = LinearRegression()


# Set the X and Y data
X = result[['rdcp']]
Y = result['g_alpha']

# Perform polynomial regression
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(X)
lr.fit(X_poly, Y)
r2_train = r2(Y, lr.predict(X_poly))

print('Coefficients: ', lr.coef_)
print('Intercept:', round(lr.intercept_,6))
print('R2: ', round(r2_train*100,2))

# Plot graph
fig = plt.figure()
ax1 = fig.add_subplot(4, 4, 1)
ax2 = fig.add_subplot(4, 4, 2)
ax3 = fig.add_subplot(4, 4, 3)
ax4 = fig.add_subplot(4, 4, 4)

plt.xlabel('Reduced Pressure [-]')
plt.ylabel('Alpha or Gamma')
plt.legend(loc='upper left')
ax1.scatter(result.loc[:, 'rdcp'], result.loc[:, 'g_alpha'], c=result['dh'], cmap ='gist_rainbow')
ax2.scatter(result.loc[:, 'rdcp'], result.loc[:,'g_gamma'], c=result['p'], cmap ='gist_rainbow')
ax3.scatter((result.loc[:, 'X_zxt']), (result.loc[:,'Y_qcal'])/result.loc[:,'g_alpha'], c=result['rdcp'], cmap ='gist_rainbow')
ax4.scatter(result.loc[:,'g_alpha'], result.loc[:,'g_gamma'], c=result['rdcp'], cmap ='gist_rainbow')

#ax1.plot(xs,ys1,'r-',lw=3)
#ax2.plot(xs,ys2,'b-',lw=3)

In [ ]:
temp_list = [(x[0], x[1]) for x in val_group]
temp_list

In [ ]:
del val_tb

In [ ]:
# alpha_avg 다항회귀
X = np.c_[prop_tb.loc[:,'rdcp'], (prop_tb.loc[:,'rdcp'])**2]
y1 = prop_tb.loc[:,'alpha']
y2 = prop_tb.loc[:, 'gamma']

# 선형 모델
model1 = LinearRegression()
model1.fit(X, y1)
model2 = LinearRegression()
model2.fit(X, y2)

xs = np.arange(min(prop_tb.rdcp),max(prop_tb.rdcp),0.02) # x 범위의 순차값 생성
ys1 = xs*model1.coef_[0] + (xs**2)*model1.coef_[1] + model1.intercept_
ys2 = xs*model2.coef_[0] + (xs**2)*model2.coef_[1] + model2.intercept_
# 2차 다항식 회귀

# 그래프
fig = plt.figure()
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)

plt.xlabel('Reduced Pressure [-]')
plt.ylabel('Alpha or Gamma')
plt.legend(loc='upper left')
ax1.scatter(prop_tb.loc[:,'rdcp'], prop_tb.loc[:,'alpha'], color = 'black')
ax2.scatter(prop_tb.loc[:,'rdcp'], prop_tb.loc[:,'gamma'], color = 'blue')

ax1.plot(xs,ys1,'r-',lw=3)
ax2.plot(xs,ys2,'b-',lw=3)


print(round(model1.coef_[0],4), round(model1.coef_[1],4), round(model1.intercept_,4))
print(round(model2.coef_[0],4), round(model2.coef_[1],4), round(model2.intercept_,4))